In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
from sklearn.model_selection import train_test_split
import cv2
from torch.optim import SGD
from torch.optim.lr_scheduler import LinearLR
from sklearn.preprocessing import OneHotEncoder

### GPU 설정

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### 데이터 가져오기

In [3]:
dir_path = '/content/drive/MyDrive/고모부_머신러닝/dogncat'
path = []
dataset_type = []
label = []
for dir_name, _, file_names in os.walk(dir_path):
    for file_name in file_names:
        file_path = dir_name + '/' + file_name
        path.append(file_path)
    
        if '/training_set' in file_path:
            dataset_type.append('train')
        elif '/test_set' in file_path:
            dataset_type.append('test')
        else:
            dataset_type.append('N/A')

        if '/cats' in file_path:
            label.append('CAT')
        elif '/dogs' in file_path:
            label.append('DOG')
        else:
            label.append('N/A')

cnd_df = pd.DataFrame({'path' : path, 'type' : dataset_type, 'label' : label})
            

In [4]:
cnd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10032 entries, 0 to 10031
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    10032 non-null  object
 1   type    10032 non-null  object
 2   label   10032 non-null  object
dtypes: object(3)
memory usage: 235.2+ KB


In [5]:
cnd_df.head(10)

,path,type,label
0,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
1,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
2,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
3,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
4,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
5,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
6,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
7,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
8,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
9,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT


In [6]:
cnd_df = cnd_df[cnd_df['path'].str.contains('.jpg')]

In [7]:
cnd_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10028 entries, 0 to 10031
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    10028 non-null  object
 1   type    10028 non-null  object
 2   label   10028 non-null  object
dtypes: object(3)
memory usage: 313.4+ KB


In [8]:
cnd_df.head()

,path,type,label
0,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
1,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
2,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
3,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
4,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT


In [9]:
# train, validation 구분
train_df, valid_df = train_test_split(cnd_df[cnd_df['type']=='train'], test_size = 0.25)

# onehotencoding
train_onehot = pd.get_dummies(train_df['label'])
valid_onehot = pd.get_dummies(valid_df['label'])

train_df = pd.concat([train_df, train_onehot], axis=1)
valid_df = pd.concat([valid_df, valid_onehot], axis=1)

# train용 path 데이터, label 데이터
train_data = train_df['path']
train_label = train_df.iloc[:, 3:].values
# train_label_indices = train_label.replace(['CAT', 'DOG'], [0,1]).values

# validation용 path, label 데이터
valid_data = valid_df['path']
valid_label = valid_df.iloc[:, 3:].values
# valid_label_indices = valid_label.replace(['CAT', 'DOG'], [0,1]).values

In [10]:
train_df

,path,type,label,CAT,DOG
2593,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,CAT,1,0
7181,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1
5046,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,CAT,1,0
2377,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,CAT,1,0
7852,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1
...,...,...,...,...,...
9465,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1
9804,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1
5387,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,CAT,1,0
9658,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1


In [11]:
train_label

array([[1, 0],
       [0, 1],
       [1, 0],
       ...,
       [1, 0],
       [0, 1],
       [1, 0]], dtype=uint8)

In [12]:
print(len(train_df))
print(len(valid_df))

6003
2002


In [13]:

# sample용 데이터

sample_dog = train_df[train_df['label'] == 'DOG'].sample(30)
sample_cat = train_df[train_df['label'] == 'CAT'].sample(30)

sample_data = pd.concat([sample_dog, sample_cat])
sample_tr, sample_val = train_test_split(sample_data, test_size=0.2)

sample_tr_data = sample_tr['path'].values # array로 dataset class에 넘겨줘야 한다...
sample_tr_label = sample_tr.iloc[:,3:].values
# sample_tr_label = sample_tr['label'].replace(['CAT', 'DOG'], [0, 1]).values

sample_val_data = sample_val['path'].values
sample_val_label = sample_tr.iloc[:,3:].values
# sample_val_label = sample_val['label'].replace(['CAT', 'DOG'], [0, 1]).values


In [14]:
sample_data.head()

,path,type,label,CAT,DOG
6824,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1
9083,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1
6365,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1
9734,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1
9135,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1


In [15]:
sample_tr_data

array(['/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/cats/cat.2708.jpg',
       '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/dogs/dog.1285.jpg',
       '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/cats/cat.3295.jpg',
       '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/dogs/dog.1415.jpg',
       '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/dogs/dog.3918.jpg',
       '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/dogs/dog.3123.jpg',
       '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/cats/cat.296.jpg',
       '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/cats/cat.2118.jpg',
       '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/cats/cat.2401.jpg',
       '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/trai

In [16]:
print(len(sample_tr_data))
print(len(sample_tr_label))

48
48


### 커스텀 데이터 셋 만들기

In [17]:
class MyDataset(Dataset):
    def __init__(self, datapath, label=None):
        super(MyDataset, self).__init__()

        self.path = datapath
        self.label = label

    def __len__(self):
        return len(self.path)

    def __getitem__(self,idx):
        img = cv2.imread(self.path[idx])
        image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (244,244))
        image = np.asarray(image, dtype=np.float32).transpose(2,0,1)
        normal_image = (image - np.amin(image)) / (np.amax(image) - np.amin(image))
        
        if self.label is not None:
            label = self.label[idx]

        return normal_image, label


In [18]:
train_dataset = MyDataset(train_data, train_label)
valid_dataset = MyDataset(valid_data, valid_label)
sample_dataset = MyDataset(sample_tr_data, sample_tr_label)

In [19]:
len(sample_dataset)

48

### 데이터로더 만들기

In [20]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)
sample_loader = DataLoader(sample_dataset, batch_size=4, shuffle=True)

In [21]:
len(sample_loader)

12

### 모델 만들기

In [22]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, kernel_size=3, padding='same')
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(5*244*244, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # print('input size :', x.shape)
        # print('input max :', torch.amax(x, dim=(1,2,3)))
        # print('input min :', torch.amin(x, dim=(1,2,3)))
        # print('input l2norm :', torch.linalg.vector_norm(x, dim=(1,2,3)))

        conv = self.conv1(x)
        # print('conv size :', conv.shape)
        # print('conv max :', torch.amax(conv, dim=(1,2,3)))
        # print('conv min :', torch.amin(conv, dim=(1,2,3)))
        # print('conv l2norm :', torch.linalg.vector_norm(conv, dim=(1,2,3)))

        conv_out = self.relu(conv)
        # print('conv_out size :', conv_out.shape)
        # print('conv_out max :', torch.amax(conv_out, dim=(1,2,3)))
        # print('conv_out min :', torch.amin(conv_out, dim=(1,2,3)))
        # print('conv_out l2norm :', torch.linalg.vector_norm(conv_out, dim=(1,2,3)))

        fc_input = conv_out.view(conv_out.size(0), -1)
        # print('fc_input size :', fc_input.shape)
        # print('fc_input max :', torch.amax(fc_input, dim=(1)))
        # print('fc_input min :', torch.amin(fc_input, dim=(1)))
        # print('fc_input l2norm :', torch.linalg.vector_norm(fc_input, dim=(1)))

        fc_logit = self.fc1(fc_input)
        # print('fc_logit size :', fc_logit.shape)
        # print('fc_logit max :', torch.amax(fc_logit, dim=(1)))
        # print('fc_logit min :', torch.amin(fc_logit, dim=(1)))
        # print('fc_logit l2norm :', torch.linalg.vector_norm(fc_logit, dim=(1)))
        # print()

        return fc_logit

In [23]:
model = CNN().to(device)
model

CNN(
  (conv1): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (relu): ReLU()
  (fc1): Linear(in_features=297680, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

### loss function

In [24]:
loss_fn = nn.KLDivLoss(reduction='batchmean').to(device)

### optimizer, lr_scheduler

In [25]:
optimizer = SGD(model.parameters(), lr=0.001)
scheduler = LinearLR(optimizer)

### Confusion matrix, Accuracy, recall, precision

In [26]:
def calculate_conf_matrix(predicted, label, class_idx):
    TP_num = 0
    TN_num = 0
    FP_num = 0
    FN_num = 0
    for i in range(len(predicted)):
        if predicted[i] == class_idx and label[i] == class_idx:
            TP_num += 1

        elif predicted[i] == class_idx and label[i] != class_idx:
            FP_num += 1
            
        elif predicted[i] != class_idx and label[i] == class_idx:
            FN_num += 1

        elif predicted[i] != class_idx and label[i] != class_idx:
            TN_num += 1

    return TP_num, TN_num, FP_num, FN_num

def calculate_accuracy(conf_matrix_list):
    try:
        accuracy = (conf_matrix_list[0] + conf_matrix_list[1]) / (conf_matrix_list[0] + conf_matrix_list[1] + conf_matrix_list[2] + conf_matrix_list[3])
    except ZeroDivisionError:
        accuracy = 0
    return accuracy

def calculate_recall(conf_matrix_list):
    try: 
        recall = conf_matrix_list[0] / (conf_matrix_list[0] + conf_matrix_list[3])
    except ZeroDivisionError:
        recall = 0
    return recall

def calculate_precision(conf_matrix_list):
    try:
        precision = conf_matrix_list[0] / (conf_matrix_list[0] + conf_matrix_list[2])
    except ZeroDivisionError:
        precision = 0
    return precision

def calculate_acc_rec_pre(conf_matrix_dict : dict):
    for class_idx in conf_matrix_dict.keys():
        accuracy = calculate_accuracy(conf_matrix_dict[class_idx])
        recall = calculate_recall(conf_matrix_dict[class_idx])
        precision = calculate_precision(conf_matrix_dict[class_idx])

        print(f'class{class_idx} >>> accuracy : {accuracy}, recall : {recall}, precision : {precision}')

    

In [28]:
epoch = 2
running_loss = 0.
running_vloss = 0.
class_num = 2
accuracy = 0.
recall = 0.
precision = 0.

for i in range(epoch):
    print()
    print(f'========= Epoch{i+1} =========')
    print()
    # train
    model.train()
    for batch_idx, data in enumerate(sample_loader):
        image_data, label_data = data
        image_data = image_data.to(device)
        label_data = label_data.type(torch.float32).to(device)
        
        optimizer.zero_grad()
        tr_output= model(image_data)
        lsf = nn.LogSoftmax()
        tr_output_lsfmax = lsf(tr_output)
        print(tr_output_lsfmax)
        loss = loss_fn(tr_output_lsfmax, label_data) # 
        print(f'batch{batch_idx+1} loss :', loss)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()

        if batch_idx + 1 == len(sample_loader):
            avg_loss = running_loss / len(sample_loader)
            print('Loss/train :', avg_loss)
            running_loss = 0.

    scheduler.step()

    print()

    # #evaluate
    # model.eval()
    # TP_TN_FP_FN = {}
    # for batch_idx, val_data in enumerate(sample_loader): # training data로 우선 evaluation해보기
    #     valid_image, valid_label = val_data
    #     valid_image = valid_image.to(device)
    #     valid_label = valid_label.to(device)
    #     val_output = model(valid_image)
    #     val_output_idx = torch.argmax(val_output, dim=1)
    #     print(f'predicted : {val_output_idx}, actual : {valid_label}')

    #     for i, class_idx in enumerate(range(class_num)):
    #             TP, TN, FP, FN = calculate_conf_matrix(val_output_idx, valid_label, class_idx)
    #             if batch_idx == 0:
    #                 TP_TN_FP_FN[class_idx] = np.array([TP, TN, FP, FN])
    #             else:
    #                 TP_TN_FP_FN[class_idx] += np.array([TP, TN, FP, FN])

    #     print(f'TP_TN_FP_FN : {TP_TN_FP_FN}')

    #     vloss = loss_fn(val_output, valid_label)
    #     running_vloss += vloss.item()
        
    #     if batch_idx + 1 == len(sample_loader):
    #         avg_vloss = running_vloss / len(sample_loader)
    #         calculate_acc_rec_pre(TP_TN_FP_FN)
    #         print('Loss/valid :', avg_vloss)
    #         # print('Accuracy :', accuracy_score(valid_label.cpu().detach().numpy(), val_output_idx.cpu().detach().numpy()))
    #         # print('Recall :', recall_score(valid_label.cpu().detach().numpy(), val_output_idx.cpu().detach().numpy()))
    #         # print('Precision :', precision_score(valid_label.cpu().detach().numpy(), val_output_idx.cpu().detach().numpy()))
    #         running_vloss = 0.



========= Epoch1 =========



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[-0.6352, -0.7546],
        [-0.6282, -0.7626],
        [-0.6326, -0.7576],
        [-0.6579, -0.7297]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
batch1 loss : tensor(0.7033, device='cuda:0', grad_fn=<DivBackward0>)
tensor([[-0.6919, -0.6944],
        [-0.7146, -0.6721],
        [-0.7102, -0.6764],
        [-0.7159, -0.6709]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
batch2 loss : tensor(0.7088, device='cuda:0', grad_fn=<DivBackward0>)
tensor([[-0.1356, -2.0653],
        [-0.1340, -2.0762],
        [-0.2293, -1.5852],
        [-0.1595, -1.9143]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
batch3 loss : tensor(1.0857, device='cuda:0', grad_fn=<DivBackward0>)
tensor([[-1.3764, -0.2910],
        [-1.3799, -0.2898],
        [-1.3227, -0.3098],
        [-1.5800, -0.2306]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
batch4 loss : tensor(1.1434, device='cuda:0', grad_fn=<DivBackward0>)
tensor([[-0.1961, -1.7257],
        [-0.0763, -2.6108],
        [-0.0510, -3

In [ ]:
input = torch.tensor([[0,1], [1,0], [1,0], [1,0]])
input

In [ ]:
input.shape